## 歷史資料回溯測試
在這份筆記中，我們將會使用昨日產生的模型來進行回測

## 資料準備

In [ ]:
%run init_model.py 'algo_ml_long_short_predict'

In [ ]:
# get S3 bucket
s3bucket=!(aws s3 ls | grep algotrading- | awk  '{print $3}')
s3bucket=s3bucket[0]
s3bucket

In [ ]:
import sys
!{sys.executable} -m pip install PyAthena

In [ ]:
import os
import sagemaker as sage
from sagemaker import get_execution_role
import datetime
from sagemaker.tensorflow import TensorFlow
import json

role = get_execution_role()
sess = sage.Session()
region = sess.boto_session.region_name

In [ ]:
import pandas as pd
from pyathena import connect
conn = connect(s3_staging_dir='s3://'+s3bucket+'/results/',
               region_name=region)

df = pd.read_sql("SELECT dt,open,high,low,close,vol FROM algo_data.hist_data_daily;", conn)
df.set_index(pd.DatetimeIndex(df['dt']),inplace=True)
del df['dt']
df.head()

In [ ]:
trainCount=int(len(df)*0.4)
dfTrain = df.iloc[:trainCount]

dfTest = df.iloc[trainCount:]
dfTest.to_csv('local/'+algo_name+'/input/data/training/data.csv')
dfTest.head()

In [ ]:
%matplotlib notebook
dfTest["close"].plot()

## 修改策略配置
在下面的單元格中，您可以調整策略的參數。

user = 名稱（可選）
long_threshold = 多頭交易的閾值（0 到 1）
short_threshold = 空頭交易的閾值（0 到 1）
profit_target_pct = 利潤目標百分比
stop_target_pct = 止損目標百分比
size = 交易的股數
提示: 改進策略的一個良好起點是修改利潤目標/止損目標以及風險/回報比。另一個選擇是通過增加閾值來減少信號的數量。

In [ ]:
%%writefile local/{algo_name}/input/config/hyperparameters.json
{ "user" : "user",
  "long_threshold" : "0.5",
  "short_threshold" : "0.5",
  "profit_target_pct" : "2.00",
  "stop_target_pct" : "1.50",
  "size" : "100"
}

In [ ]:
%run update_config.py $algo_name

## 修改策略代碼
下面的程式碼，可能會需要不斷修改測試來改進我們的策略

這段程式碼是一個用於量化交易的策略，使用了Backtrader庫，並結合了深度學習模型來進行交易決策。以下是對程式碼的詳細解釋：
MyStrategy 類別是策略的主體，繼承自 StrategyTemplate。
在 __init__ 方法中，策略初始化設定，包括讀取模型（'model_long_short_predict.h5'），並設定一些策略參數，如長期和短期閾值、交易量、利潤目標百分比、停損目標百分比等。
在 add_data 方法中，向策略添加交易數據。通過讀取 MyStrategy.TRAIN_FILE 中的CSV數據來添加。
next 方法是策略的核心，它在每個交易日呼叫一次。在這裡，策略計算技術指標（SMA和ROC）並準備模型的輸入數據。
inputRec 是用於存儲輸入數據的列表，包括日期（dt）和收盤價（close）。
接下來是計算SMA（簡單移動平均）和ROC（變化率）技術指標，並將它們添加到 inputRec 中。
接下來，將 inputRec 轉換為NumPy數組，以供深度學習模型進行預測。
模型預測兩個值，分別代表長期和短期的信號（tLong和tShort）。
如果策略目前沒有持倉（position），則根據預測的信號和閾值，判斷是進行多頭交易（buy）還是空頭交易（sell）。如果預測的tLong大於長期閾值，則執行多頭交易；如果預測的tShort大於短期閾值，則執行空頭交易。
如果策略目前有持倉，則根據限價價格（limitPrice）和停損價格（stopPrice）來判斷是否應該平倉。
總之，這個策略將深度學習模型的預測信號與技術指標結合在一起，來做出交易決策，並根據預定的利潤目標和停損目標來進行風險管理。

In [ ]:
%%writefile model/{algo_name}.py
import backtrader as bt
from algo_base import *
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

class MyStrategy(StrategyTemplate):

    def __init__(self):
        super(MyStrategy, self).__init__()
        self.config["long_threshold"]=float(self.config["long_threshold"])
        self.config["short_threshold"]=float(self.config["short_threshold"])
        self.config["size"]=int(self.config["size"])
        self.config["profit_target_pct"]=float(self.config["profit_target_pct"])
        self.config["stop_target_pct"]=float(self.config["stop_target_pct"])

        self.order=None
        self.orderPlaced=False
                                
        self.model = load_model('model_long_short_predict.h5')
        
        # input / indicators
        self.repeatCount=15
        self.repeatStep=1
        
        self.profitTarget=self.config["profit_target_pct"]/100.0
        self.stopTarget=self.config["stop_target_pct"]/100.0
        self.size=self.config["size"]
         
        self.sma=[]
        self.roc=[]
        
        self.hData=["dt"]
        self.hData.append("close") 
        for a in range(0,self.repeatCount):
            tp=(a+1)*self.repeatStep+1
            self.hData.append("sma"+str(tp))
            self.sma.append(bt.talib.SMA(self.data, timeperiod=tp, plot=False))
        for a in range(0,self.repeatCount):
            tp=(a+1)*self.repeatStep+1
            self.hData.append("roc"+str(tp))
            self.roc.append(bt.talib.ROC(self.data, timeperiod=tp, plot=False))

    def init_broker(broker):
        broker.setcash(100000.0)
        broker.setcommission(commission=0.0) 
        
    def add_data(cerebro):
        data = btfeeds.GenericCSVData(
            dataname=MyStrategy.TRAIN_FILE,
            dtformat=('%Y-%m-%d'),
            timeframe=bt.TimeFrame.Days,
            datetime=0,
            time=-1,
            high=2,
            low=3,
            open=1,
            close=4,
            volume=5,
            openinterest=-1
        )
        cerebro.adddata(data)

    def next(self):
        super(MyStrategy, self).next()
        
        dt=self.datas[0].datetime.datetime(0)
        cl=self.dataclose[0]
        inputRec=[]                

        #open
        inputRec0=[]
        inputRec0.append(cl)

        #sma
        for a in range(0,self.repeatCount):
            if math.isnan(self.sma[a][0]):
                inputRec0.append(cl)
            else:
                inputRec0.append(self.sma[a][0])

        m1=min(inputRec0)
        m2=max(inputRec0)
        for a in inputRec0:
            if m2-m1==0:
                inputRec.append(0)
            else:
                inputRec.append((a-m1)/(m2-m1))

        #roc
        for a in range(0,self.repeatCount):
            if math.isnan(self.roc[a][0]):
                inputRec.append(0)
            else:
                inputRec.append(self.roc[a][0])

        mX=[]
        mX.append(np.array(inputRec))
        dataX=np.array(mX)
        #print("dataX=%s" % dataX)

        # *** ML prediction ***
        mY=self.model.predict(dataX)
        #print("mY=%s" % mY)
        tLong=mY[0][0]
        tShort=mY[0][1]
        #print("[%s]:long=%s,short=%s" % (dt,tLong,tShort))
        if not self.position:
            fLong=(tLong>self.config["long_threshold"]) 
            fShort=(tShort>self.config["short_threshold"])
            if fLong:
                self.order=self.buy(size=self.size)
                self.limitPrice=cl+self.profitTarget*cl
                self.stopPrice=cl-self.stopTarget*cl
            elif fShort:
                self.order=self.sell(size=self.size)                    
                self.limitPrice=cl-self.profitTarget*cl
                self.stopPrice=cl+self.stopTarget*cl

        if self.position:
            if self.position.size>0:
                if cl>=self.limitPrice or cl<=self.stopPrice:
                    self.order=self.sell(size=self.size)
            elif self.position.size<0:
                if cl<=self.limitPrice or cl>=self.stopPrice:
                    self.order=self.buy(size=self.size)

## 執行本地端回溯測試
第一次因為要初始化 docker build image，時間會比較久

In [ ]:
#Build Local Algo Image
!docker build -t $algo_name .
!docker run -v $(pwd)/local/$algo_name:/opt/ml --rm $algo_name train

In [ ]:
from IPython.display import Image
Image(filename='local/'+algo_name+'/model/chart.png')

## 接著我們可以重複 修改策略配置 ～ 執行本地端回溯測試 步驟來調整我們的策略
調整完畢之後可以將 docker image 推送到 ECR 上準備部署到 SageMaker 的 ECS 上

In [ ]:
#Deploy Algo Image to ECS
!./build_and_push.sh $algo_name

這段程式碼是用於在Amazon SageMaker上運行遠程的前測試（Remote Forwardtest）的操作。

引用SageMaker相關的庫和工具：

import sagemaker as sage: 引用SageMaker的Python SDK。
from sagemaker import get_execution_role: 從SageMaker庫中引入用於獲取執行角色的函數。
from sagemaker.estimator import Estimator: 引入SageMaker估算器，用於定義和訓練機器學習模型。
獲取SageMaker執行角色和建立SageMaker會話：

role = get_execution_role(): 使用get_execution_role函數獲取SageMaker的執行角色，該角色賦予了運行SageMaker作業的權限。
sess = sage.Session(): 建立SageMaker會話，用於管理SageMaker作業。

上傳訓練數據到S3存儲桶：
WORK_DIRECTORY定義了本地訓練數據的目錄。
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix='data')：使用SageMaker會話的upload_data方法將本地訓練數據上傳到S3存儲桶中，同時指定了上傳的目標路徑。
讀取並設置模型的 hyperparameters 配置：

conf_file定義了超參數配置文件的路徑。
使用 json.load(f) 讀取超參數配置文件，並將其存儲在config變數中。
配置 SageMaker 估算器：

prefix 和 job_name 用於定義作業名稱。
account 和 region用於獲取AWS帳戶和區域信息。
image 指定了 Docker 映像的 URI，其中包含了要運行的機器學習算法。
algo = sage.estimator.Estimator(...): 創建一個SageMaker估算器，配置了執行訓練作業所需的所有參數，包括執行角色、實例數量、實例類型、輸出路徑、超參數等。
訓練模型：

algo.fit(data_location): 使用估算器的fit方法開始訓練模型，並將訓練數據從S3存儲桶中讀取。訓練完成後，模型的結果和輸出將被存儲在指定的S3存儲桶中。

In [ ]:
#Run Remote Forwardtest via SageMaker
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator 

role = get_execution_role()
sess = sage.Session()

WORK_DIRECTORY = 'local/'+algo_name+'/input/data/training'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix='data')
print(data_location)

conf_file='local/'+algo_name+'/input/config/hyperparameters.json'
with open(conf_file, 'r') as f:
    config = json.load(f)
#config['sim_data']='True'
print(config)

prefix=algo_name
job_name=prefix.replace('_','-')

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

algo = sage.estimator.Estimator(
    image_uri=image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
    base_job_name=job_name,
    hyperparameters=config,
    metric_definitions=[
        {
            "Name": "algo:pnl",
            "Regex": "Total PnL:(.*?)]"
        },
        {
            "Name": "algo:sharpe_ratio",
            "Regex": "Sharpe Ratio:(.*?),"
        }
    ])
algo.fit(data_location)

In [ ]:
#Get Algo Metrics
from sagemaker.analytics import TrainingJobAnalytics

latest_job_name = algo.latest_training_job.job_name
metrics_dataframe = TrainingJobAnalytics(training_job_name=latest_job_name).dataframe()
metrics_dataframe